### Python's Assignment Expressions

Python 3.8 introduced the `:=` operator (the so-called "walrus" operator) for assignment expressions. (see [PEP 572](https://peps.python.org/pep-0572/))

What the heck is an assignment expression?

Basically, it is an expression (so it returns a value when evaluated), just like something like this:

In [1]:
10 + (20 * 30) // 3

210

**And** it is also an assigment at the same time.

Now, the walrus operator does not replace the regular assignment operator, we still do assignments the same way, but it allows us to embed assignments **inside** other expressions.

So, something like this is not allowed:

```
a := 20 + 30
```

Technically you could write it this way, but this is highly discouraged and should not be done!

In [2]:
(x := 20 + 30)
x

50

So we still use regular assignments, but now we can do something like this:

In [3]:
a = (b := 2 * 3) % 5

Not only do we have a calculated value for `a`:

In [4]:
a

1

But now we also have a new variable `b`:

In [5]:
b

6

This can be quite useful to define long formulas and extract sub-calculations into variables - either for debugging, or for easily extracting the sub-calculations for further processing.

There are quite a few more contexts where the assignment expression syntax comes in quite useful.

Consider this example, where we have a list of strings, and we want to transform that list into a list of strings that contain all the unique characters of each string but only for cases where the number of unique characters is greater than `1`.

In [6]:
l = ["a", "aa", "aaa", "ab", "aab", "aabb"]

We could use a comprehension to do this:

In [7]:
result = [set(el) for el in l if len(set(el)) > 1]
result

[{'a', 'b'}, {'a', 'b'}, {'a', 'b'}]

But you'll notice that we had to calculate `set(el)` **twice** - once in the `if` clause, and once in the value we want for each resulting element.

Instead, we could use the walrus operator this way:

In [8]:
result = [chars for el in l if len(chars := set(el)) > 1]
result

[{'a', 'b'}, {'a', 'b'}, {'a', 'b'}]

Since the `if` clause gets evaluated first, we can stick our assignment expression there, and then use that variable elsewhere in our comprehension.

Now this simple example probably does not require this approach - we really don't gain much by using the walrus operator here.

But consider the case where both the `if` clause expression and the elements we want returned are based on some long running function:

In [9]:
import time

def transform(x):
    time.sleep(0.2)
    sign = 1 if x % 2 == 0 else -1
    return sign *  x ** 2

In [10]:
l = [1, 2, 3, 4, 5, 6, 7, 8, 10]

In [11]:
start = time.perf_counter()
result = [transform(x) for x in l if transform(x) > 0]
end = time.perf_counter()
print(result)
print(f"elapsed: {end - start:.1f} seconds")

[4, 16, 36, 64, 100]
elapsed: 2.9 seconds


You'll notice that in this case we had to calculate transform **twice**. Now, let's make this a bit more efficient by using the walrus operator:

In [12]:
start = time.perf_counter()
result = [val for x in l if (val := transform(x)) > 0]
end = time.perf_counter()
print(result)
print(f"elapsed: {end - start:.1f} seconds")

[4, 16, 36, 64, 100]
elapsed: 1.8 seconds


Here's another scenario where the assignment expression can be useful too, building up a list, or dictionary of values, where one value might be based on a "previous" one:

In [13]:
l = [
    (start := 10),
    (intermediate := start + 10),
    start + intermediate
]
l

[10, 20, 30]

This works with dictionaries too:

In [14]:
d = {
    "start": (start := 10),
    "intermediate": (intermediate := start + 10),
    "last": start + intermediate
}
d

{'start': 10, 'intermediate': 20, 'last': 30}

Let's look at a slightly more realistic example - suppose we have a deck of cards, and we want to randomly select hands of 5 cards from the deck, but only retain the hands that have at least one card in each suit, and for each hand we want to also calculate the frequency of cards in each suit.

Let's do this step by step.

We build the deck first:

In [15]:
suits = list("SHDC")  # spades, hearts, diamonds, clubs
faces = list("23456789") + ["10"] + list("JQKA")

deck = [(f, s) for s in suits for f in faces]

In [16]:
deck

[('2', 'S'),
 ('3', 'S'),
 ('4', 'S'),
 ('5', 'S'),
 ('6', 'S'),
 ('7', 'S'),
 ('8', 'S'),
 ('9', 'S'),
 ('10', 'S'),
 ('J', 'S'),
 ('Q', 'S'),
 ('K', 'S'),
 ('A', 'S'),
 ('2', 'H'),
 ('3', 'H'),
 ('4', 'H'),
 ('5', 'H'),
 ('6', 'H'),
 ('7', 'H'),
 ('8', 'H'),
 ('9', 'H'),
 ('10', 'H'),
 ('J', 'H'),
 ('Q', 'H'),
 ('K', 'H'),
 ('A', 'H'),
 ('2', 'D'),
 ('3', 'D'),
 ('4', 'D'),
 ('5', 'D'),
 ('6', 'D'),
 ('7', 'D'),
 ('8', 'D'),
 ('9', 'D'),
 ('10', 'D'),
 ('J', 'D'),
 ('Q', 'D'),
 ('K', 'D'),
 ('A', 'D'),
 ('2', 'C'),
 ('3', 'C'),
 ('4', 'C'),
 ('5', 'C'),
 ('6', 'C'),
 ('7', 'C'),
 ('8', 'C'),
 ('9', 'C'),
 ('10', 'C'),
 ('J', 'C'),
 ('Q', 'C'),
 ('K', 'C'),
 ('A', 'C')]

We can select 5 cards randomly from this deck by using the random module's `choices` function:

In [17]:
import random

random.seed(0)  # setting seed for repeatability

In [18]:
hand = random.choices(deck, k=5)
hand

[('6', 'C'), ('2', 'C'), ('10', 'H'), ('2', 'H'), ('2', 'D')]

Now, for this "hand" we want to count the number of cards for each suit:

In [19]:
from collections import defaultdict

In [20]:
def suit_counts(hand):
    result = defaultdict(int)
    for face, suit in hand:
        result[suit] += 1
    return dict(result)

In [21]:
suit_counts(hand)

{'C': 2, 'H': 2, 'D': 1}

But what we really want it just to know if the hand has at least one card of each suit:

In [22]:
def hand_has_all_suits(suit_counts):
    return len(suit_counts) == 4

In [23]:
hand_has_all_suits(suit_counts(hand))

False

In [24]:
hand = [('1', 'S'), ('1', 'H'), ('1', 'D'), ('1', 'C')]
counts = suit_counts(hand)
print(counts)
print(hand_has_all_suits(counts))

{'S': 1, 'H': 1, 'D': 1, 'C': 1}
True


Ok, so now we have everything we need to generate a list of randomly selected hands, filter out the ones that do not contain at least one card from each suit, and also calculate the frequency distribution of the suits in each hand.

Again, let's built this step by step:

In [25]:
hands = [
    random.choices(deck, k=5)
    for _ in range(10)
]

In [26]:
hands

[[('10', 'H'), ('3', 'C'), ('4', 'H'), ('K', 'H'), ('6', 'D')],
 [('10', 'C'), ('2', 'D'), ('3', 'H'), ('2', 'C'), ('8', 'D')],
 [('2', 'H'), ('10', 'C'), ('A', 'C'), ('5', 'C'), ('9', 'C')],
 [('5', 'H'), ('K', 'D'), ('9', 'C'), ('J', 'D'), ('K', 'H')],
 [('7', 'S'), ('J', 'H'), ('7', 'D'), ('10', 'C'), ('K', 'C')],
 [('K', 'H'), ('7', 'C'), ('2', 'H'), ('4', 'C'), ('4', 'D')],
 [('2', 'S'), ('K', 'D'), ('9', 'H'), ('5', 'C'), ('10', 'D')],
 [('2', 'S'), ('A', 'H'), ('8', 'C'), ('A', 'S'), ('5', 'H')],
 [('8', 'C'), ('J', 'S'), ('5', 'D'), ('A', 'S'), ('K', 'C')],
 [('4', 'C'), ('Q', 'H'), ('6', 'S'), ('5', 'H'), ('2', 'D')]]

Let's add in the filter clause - without using the walrus operator. One thing to note is that for each iteration, we **need** to be using the same hand in every calculation (suit counts, and filter clause), so something like this will simply not work correctly:

In [27]:
# yes this is wrong, I know!
random.seed(0)
hands = [
    random.choices(deck, k=5)
    for _ in range(10)
    if hand_has_all_suits(suit_counts(random.choices(deck, k=5)))
]
print([suit_counts(hand) for hand in hands])

[{'H': 2, 'C': 2, 'D': 1}, {'S': 2, 'H': 2, 'C': 1}, {'C': 1, 'S': 1, 'D': 3}]


As you can see, this is not working right - because the hand being returned in each iteration is a different hand than the one we are using in the `if` clause. And the bug would be compounded if we additionaly returned the hand frequency distribution:

In [28]:
random.seed(0)
hands = [
    (random.choices(deck, k=5), suit_counts(random.choices(deck, k=5)))
    for _ in range(10)
    if hand_has_all_suits(suit_counts(random.choices(deck, k=5)))
]
for hand in hands:
    print(suit_counts(hand[0]))
    print(hand[1])
    print('-' * 20)

{'H': 2, 'C': 2, 'D': 1}
{'S': 1, 'D': 2, 'H': 1, 'C': 1}
--------------------
{'C': 1, 'S': 1, 'D': 3}
{'C': 2, 'D': 3}
--------------------


Doing this using a list comprehension without the walrus operator takes a bit more work, and we could do it this way:

In [29]:
from itertools import repeat

In [30]:
random.seed(0)
hands = [
    hand
    for hand in [random.choices(deck, k=5) for _ in range(10)]
]
hands

[[('6', 'C'), ('2', 'C'), ('10', 'H'), ('2', 'H'), ('2', 'D')],
 [('10', 'H'), ('3', 'C'), ('4', 'H'), ('K', 'H'), ('6', 'D')],
 [('10', 'C'), ('2', 'D'), ('3', 'H'), ('2', 'C'), ('8', 'D')],
 [('2', 'H'), ('10', 'C'), ('A', 'C'), ('5', 'C'), ('9', 'C')],
 [('5', 'H'), ('K', 'D'), ('9', 'C'), ('J', 'D'), ('K', 'H')],
 [('7', 'S'), ('J', 'H'), ('7', 'D'), ('10', 'C'), ('K', 'C')],
 [('K', 'H'), ('7', 'C'), ('2', 'H'), ('4', 'C'), ('4', 'D')],
 [('2', 'S'), ('K', 'D'), ('9', 'H'), ('5', 'C'), ('10', 'D')],
 [('2', 'S'), ('A', 'H'), ('8', 'C'), ('A', 'S'), ('5', 'H')],
 [('8', 'C'), ('J', 'S'), ('5', 'D'), ('A', 'S'), ('K', 'C')]]

And we can then add in out other calculations:

In [31]:
random.seed(0)
hands = [
    (hand, suit_counts(hand))
    for hand in [random.choices(deck, k=5) for _ in range(10)]
    if hand_has_all_suits(suit_counts(hand))
]    

In [32]:
for hand in hands:
    print(suit_counts(hand[0]))
    print(hand[1])
    print('-' * 20)

{'S': 1, 'H': 1, 'D': 1, 'C': 2}
{'S': 1, 'H': 1, 'D': 1, 'C': 2}
--------------------
{'S': 1, 'D': 2, 'H': 1, 'C': 1}
{'S': 1, 'D': 2, 'H': 1, 'C': 1}
--------------------


But, you'll notice that we are still calcualting `suit_counts` twice. 

Instead, we can turn to the walrus operator to help us avoid that, **and** simplify the comprehension:

In [33]:
random.seed(0)
hands = [
    (hand, counts)
    for _ in range(10)
    if hand_has_all_suits(counts := suit_counts(hand := random.choices(deck, k=5)))
] 

In [34]:
for hand in hands:
    print(suit_counts(hand[0]))
    print(hand[1])
    print('-' * 20)

{'S': 1, 'H': 1, 'D': 1, 'C': 2}
{'S': 1, 'H': 1, 'D': 1, 'C': 2}
--------------------
{'S': 1, 'D': 2, 'H': 1, 'C': 1}
{'S': 1, 'D': 2, 'H': 1, 'C': 1}
--------------------


Yet another useful application of an assignment expression is with `while` loops.

Consider this example where we are receiving user input, and want to keep prompting the user for an input until a valid input has been received.

In [35]:
def validate_flag(x):
    return x.casefold() in {'y', 'n'}
        
is_valid = False
while not is_valid:
    response = input("Do you want to abort? (Y/n): ")
    is_valid = validate_flag(response)
    if not is_valid:
        print("Please enter Y/y or N/n")
print(f"You decided: {response.upper()}")

Do you want to abort? (Y/n): n
You decided: N


We could rewrite this without keeping that `is_valid` flag around altogether, using an assignment expression:

In [36]:
while not validate_flag(response := input("Do you want to abort? (Y/n): ")):
    print("Please enter Y/y or N/n")
print(f"You decided: {response.upper()}")

Do you want to abort? (Y/n): n
You decided: N


Another area where the assignment operation comes in useful is when using functions such as `any` or `all`.
These functions will test whether at least one element or every element of an iterable is `True`:

In [37]:
any([True, False, False])

True

In [38]:
all([True, False, False])

False

Let's look at this example where we have a list of strings, and we want to ensure that every string in the list is at least two characters:

In [39]:
l = ['aa', 'bb', 'c', 'dd']

In [40]:
all(len(s) > 1 for s in l)

False

This is great, but what if we actually want to know the first element that caused `all` to return `False`?

Normally we might go about it this way:

In [41]:
is_ok = True
example = None
for s in l:
    if len(s) <= 1:
        is_ok = False
        example = s
        break

if not is_ok:
    print(f"Invalid string found: {example}")

Invalid string found: c


But look at how we can leverage the walrus operator:

In [42]:
is_ok = all(len(example := s) > 1 for s in l)

if not is_ok:
    print(f"Invalid string found: {example}")

Invalid string found: c


Here's another fun way to use the walrus operator - for accumulations.

Suppose that given a list of numbers, we want to produce a cumulative sum of the squares of each number:


We could certainly do it this way:

In [43]:
data = [1, 2, 3, 4, 5, 6]
sums = []
sum_ = 0
for num in data:
    sum_ = sum_ + num ** 2 
    sums.append(sum_)

sums

[1, 5, 14, 30, 55, 91]

But we could also use an assignment expression here:

In [44]:
sum_ = 0
[(sum_ := sum_ + num ** 2) for num in data]

[1, 5, 14, 30, 55, 91]

We could even use it to calculate consecutive factorials without using recursion:

In [45]:
prod = 1
[(prod := prod * num) for num in range(1, 6)]

[1, 2, 6, 24, 120]

Now there are some limitations to where you can use this new operator.

You cannot use it in iterable unpacking:

In [46]:
t = 1, 2, 3

a, b, c = t

You also cannot use it in augmented assignments such as `+=`, `*=`, etc. See the example we just did with factorials and sums - we had to use the something like `sum_ := sum_ + x`

You cannot use it to create an attribute on a class and return that value at the same time:

In [47]:
class Person:
    pass

In [48]:
p = Person()
p.name = 'Fred'
p.name

'Fred'

In [49]:
p = Person()
p.name := 'Fred'

SyntaxError: cannot use assignment expressions with attribute (2999376119.py, line 2)

There are a few more cases where you will not be able to use that operator - but most of the times these are cases where it would not make sense anyway (and hence why those cases are not supported). See the PEP linked above for more details.

In particular I strongly urge you to read the Appendix A section of the PEP for some rational guidelines on when to use assignment expressions - it can be too easy to write code that's hard to understand!

[PEP 572 - Appendix A](https://peps.python.org/pep-0572/#appendix-a-tim-peters-s-findings)

As you work with this new operator you will likely run across parts of your code where you'll quickly realize that this new operator could come in handy.